In [1]:
import json
import requests
import pandas as pd
import io

# Step 1 Read Meta Data  & End Points 

In [2]:
def get_ids(start, end):
    years = [str(x) for x in range(start, end+1)]
    ids = {"General": [], "Research": [], "Ownership": []}
    url = 'https://openpaymentsdata.cms.gov/api/1/metastore/schemas/dataset/items?show-reference-ids'
    data = requests.get(url).json()
    for d in data:
        split_title = d['title'].split(" ")
        identifier = d['identifier']
        if split_title[0] in years:
            if split_title[1] in ids:
                ids[split_title[1]].append(identifier)
    return ids

In [3]:
ids = get_ids(2020, 2021)

In [4]:
usecols=["Date_of_Payment", "Product_Category_or_Therapeutic_Area_1", "Recipient_State", "Recipient_City", "Covered_Recipient_Type", "Total_Amount_of_Payment_USDollars"]

In [5]:
def get_datasets(id_list):
    for category, category_ids in id_list.items():
        for category_id in category_ids:
            url = 'https://openpaymentsdata.cms.gov/api/1/metastore/schemas/dataset/items/{}'.format(category_id)
            data = requests.get(url).json()
            download = data["distribution"][0]["downloadURL"]
            try:
                response = requests.get(download)
                df = pd.read_csv(io.BytesIO(response.content), nrows=10)
                memory_usage = df.memory_usage(deep=True)
                # Sum up the memory usage of all columns
                total_memory_usage = memory_usage.sum()
                total_memory_usage_gb = total_memory_usage / (1024 ** 3)
                # Print the total memory usage
                print(category)
                print("Total memory usage:", total_memory_usage_gb)
                print("\n")
            except Exception as e:
                print("Failed to download:", download, "Error:", e)

In [6]:
df = get_datasets(ids)

General
Total memory usage: 2.4804845452308655e-05


General
Total memory usage: 2.5272369384765625e-05


Research
Total memory usage: 4.1046179831027985e-05


Research
Total memory usage: 4.139076918363571e-05


Ownership
Total memory usage: 1.2828968465328217e-05


Ownership
Total memory usage: 1.2950040400028229e-05




In [8]:
df